# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# magic word for producing visualizations in notebook
%matplotlib inline

## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

In [2]:
azdias = pd.read_csv('Data/Udacity_AZDIAS_052018.csv', sep=';')
customers = pd.read_csv('Data/Udacity_CUSTOMERS_052018.csv', sep=';')

/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#get to know the features of azdias
azdias.head()

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,...,VHN,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,910215,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,1,2
1,910220,-1,9.0,0.0,NaN,NaN,NaN,NaN,21.0,11.0,...,4.0,8.0,11.0,10.0,3.0,9.0,4.0,5,2,1
2,910225,-1,9.0,17.0,NaN,NaN,NaN,NaN,17.0,10.0,...,2.0,9.0,9.0,6.0,3.0,9.0,2.0,5,2,3
3,910226,2,1.0,13.0,NaN,NaN,NaN,NaN,13.0,1.0,...,0.0,7.0,10.0,11.0,NaN,9.0,7.0,3,2,4
4,910241,-1,1.0,20.0,NaN,NaN,NaN,NaN,14.0,3.0,...,2.0,3.0,5.0,4.0,2.0,9.0,3.0,4,1,3


In [4]:
azdias.info()
azdias.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891221 entries, 0 to 891220
Columns: 366 entries, LNR to ALTERSKATEGORIE_GROB
dtypes: float64(267), int64(93), object(6)
memory usage: 2.4+ GB


(891221, 366)

According to the shape information, there are 891,221 rows/individuals in the dataset. Each individual has 366 features. 


## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

## Data Cleaning

1.1 From the DIAS Attributes excel sheet, I created a csv file called Data_NaN that denotes values that mean unknown cases in each column. From Data_NaN I create a corresponding dictionary.

In [5]:
pd_NaN = pd.read_csv('Data/Data_NaN.csv', sep=',', header=0)
pd_NaN.head()

,Attribute,Value
0,AGER_TYP,-1
1,ALTERSKATEGORIE_GROB,"-1, 0"
2,ALTER_HH,0
3,ANREDE_KZ,"-1, 0"
4,BALLRAUM,-1


In [6]:
pd_NaN['Value'] = pd_NaN['Value'].apply(lambda x: x.split(","))
pd_NaN['Value'] = pd_NaN['Value'].apply(lambda x: list(map(int,x)))

In [7]:
dict_NaN = dict(zip(pd_NaN['Attribute'], pd_NaN['Value']))
print(dict_NaN)

{'AGER_TYP': [-1], 'ALTERSKATEGORIE_GROB': [-1, 0], 'ALTER_HH': [0], 'ANREDE_KZ': [-1, 0], 'BALLRAUM': [-1], 'CAMEO_DEUG_2015': [-1], 'CAMEO_INTL_2015': [-1], 'CJT_GESAMTTYP': [0], 'EWDICHTE': [-1], 'FINANZTYP': [-1], 'FINANZ_ANLEGER': [-1], 'FINANZ_HAUSBAUER': [-1], 'FINANZ_MINIMALIST': [-1], 'FINANZ_SPARER': [-1], 'FINANZ_UNAUFFAELLIGER': [-1], 'FINANZ_VORSORGER': [-1], 'GEBAEUDETYP': [-1, 0], 'HEALTH_TYP': [-1], 'HH_EINKOMMEN_SCORE': [-1, 0], 'INNENSTADT': [-1], 'KBA05_ALTER1': [-1, 9], 'KBA05_ALTER2': [-1, 9], 'KBA05_ALTER3': [-1, 9], 'KBA05_ALTER4': [-1, 9], 'KBA05_ANHANG': [-1, 9], 'KBA05_ANTG1': [-1], 'KBA05_ANTG2': [-1], 'KBA05_ANTG3': [-1], 'KBA05_ANTG4': [-1], 'KBA05_AUTOQUOT': [-1, 9], 'KBA05_BAUMAX': [-1, 0], 'KBA05_CCM1': [-1, 9], 'KBA05_CCM2': [-1, 9], 'KBA05_CCM3': [-1, 9], 'KBA05_CCM4': [-1, 9], 'KBA05_DIESEL': [-1, 9], 'KBA05_FRAU': [-1, 9], 'KBA05_GBZ': [-1, 0], 'KBA05_HERST1': [-1, 9], 'KBA05_HERST2': [-1, 9], 'KBA05_HERST3': [-1, 9], 'KBA05_HERST4': [-1, 9], 'KBA05_

In [8]:
#double check whether any item in dictionary keys is not in the column names 
list_1 = list(dict_NaN.keys())
list_2 = list(azdias.columns.values)
diff_list = np.setdiff1d(list_1,list_2)
print(diff_list)  



[]


1.2 Convert the unknown values in azdias into NaN. 

In [9]:
azdias_1 = azdias[:10000].copy()

In [10]:
def convert_unknown_to_NaN(df, unknown = dict_NaN):
    '''
    It maps unknown values by column and transform them to nan values.
    
    Input:
    df: original dataframe;
    unkown: dictionary mapping columns and their respective unknown values.
    
    Output:Processed dataframe that maps unknown values to NaN   
    '''
    df_1 = df.copy()
    for col in list(unknown.keys()):
        df_1.loc[:, col] = df_1.loc[:, col].apply(lambda x: np.nan if x in unknown[col] else x)
    return df_1 

In [11]:
azdias_NaN = convert_unknown_to_NaN(df=azdias_1)

In [12]:
azdias_NaN.head()

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,...,VHN,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,910215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,1,2
1,910220,NaN,9.0,NaN,NaN,NaN,NaN,NaN,21.0,11.0,...,4.0,8.0,11.0,10.0,3.0,9.0,4.0,5,2,1
2,910225,NaN,9.0,17.0,NaN,NaN,NaN,NaN,17.0,10.0,...,2.0,9.0,9.0,6.0,3.0,9.0,2.0,5,2,3
3,910226,2.0,1.0,13.0,NaN,NaN,NaN,NaN,13.0,1.0,...,0.0,7.0,10.0,11.0,NaN,9.0,7.0,3,2,4
4,910241,NaN,1.0,20.0,NaN,NaN,NaN,NaN,14.0,3.0,...,2.0,3.0,5.0,4.0,2.0,9.0,3.0,4,1,3


1.3 Find the features that have NaN above 15% and delete the features. Here due to the constraint of memory, I chose first 10,000 data to decide the NaN percentage. 

In [13]:
thresh = int(0.85*10000)
azdias_NaN_drop = azdias_NaN.dropna(axis=1, thresh=thresh, inplace=False)
azdias_NaN_drop.head()

,LNR,AKT_DAT_KL,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,ANZ_KINDER,ANZ_PERSONEN,ANZ_STATISTISCHE_HAUSHALTE,ANZ_TITEL,ARBEIT,BALLRAUM,...,VERS_TYP,VHA,VK_DHT4A,VK_DISTANZ,VK_ZG11,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,910215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,1,2
1,910220,9.0,11.0,0.0,0.0,2.0,12.0,0.0,3.0,6.0,...,2.0,0.0,8.0,11.0,10.0,9.0,4.0,5,2,1
2,910225,9.0,10.0,0.0,0.0,1.0,7.0,0.0,3.0,2.0,...,1.0,0.0,9.0,9.0,6.0,9.0,2.0,5,2,3
3,910226,1.0,1.0,0.0,0.0,0.0,2.0,0.0,2.0,4.0,...,1.0,1.0,7.0,10.0,11.0,9.0,7.0,3,2,4
4,910241,1.0,3.0,0.0,0.0,4.0,3.0,0.0,4.0,2.0,...,2.0,0.0,3.0,5.0,4.0,9.0,3.0,4,1,3


Find the columns that were dropped due to high NaN percentage

In [14]:
list_1 = list(azdias_NaN_drop.columns.values)
list_2 = list(azdias_NaN.columns.values)
column_NaN = np.setdiff1d(list_2,list_1).tolist()
print(column_NaN)

['AGER_TYP', 'ALTERSKATEGORIE_FEIN', 'ALTER_HH', 'ALTER_KIND1', 'ALTER_KIND2', 'ALTER_KIND3', 'ALTER_KIND4', 'D19_BANKEN_ONLINE_QUOTE_12', 'D19_GESAMT_ONLINE_QUOTE_12', 'D19_KONSUMTYP', 'D19_LETZTER_KAUF_BRANCHE', 'D19_LOTTO', 'D19_SOZIALES', 'D19_TELKO_ONLINE_QUOTE_12', 'D19_VERSAND_ONLINE_QUOTE_12', 'D19_VERSI_ONLINE_QUOTE_12', 'EXTSEL992', 'KBA05_ALTER1', 'KBA05_ALTER2', 'KBA05_ALTER3', 'KBA05_ALTER4', 'KBA05_ANHANG', 'KBA05_ANTG1', 'KBA05_ANTG2', 'KBA05_ANTG3', 'KBA05_ANTG4', 'KBA05_AUTOQUOT', 'KBA05_BAUMAX', 'KBA05_CCM1', 'KBA05_CCM2', 'KBA05_CCM3', 'KBA05_CCM4', 'KBA05_DIESEL', 'KBA05_FRAU', 'KBA05_GBZ', 'KBA05_HERST1', 'KBA05_HERST2', 'KBA05_HERST3', 'KBA05_HERST4', 'KBA05_HERST5', 'KBA05_KRSAQUOT', 'KBA05_KRSHERST1', 'KBA05_KRSHERST2', 'KBA05_KRSHERST3', 'KBA05_KRSKLEIN', 'KBA05_KRSOBER', 'KBA05_KRSVAN', 'KBA05_KRSZUL', 'KBA05_KW1', 'KBA05_KW2', 'KBA05_KW3', 'KBA05_MAXAH', 'KBA05_MAXBJ', 'KBA05_MAXHERST', 'KBA05_MAXSEG', 'KBA05_MAXVORB', 'KBA05_MOD1', 'KBA05_MOD2', 'KBA05_MOD3'

1.4 Find the columns of object dtypes: either convert the column into int/float dtypes, or drop the column

In [15]:
azdias_object = azdias_NaN_drop.select_dtypes(include='object')

azdias_object.head()

,CAMEO_DEU_2015,CAMEO_DEUG_2015,CAMEO_INTL_2015,EINGEFUEGT_AM,OST_WEST_KZ
0,NaN,NaN,NaN,NaN,NaN
1,8A,8,51,1992-02-10 00:00:00,W
2,4C,4,24,1992-02-12 00:00:00,W
3,2A,2,12,1997-04-21 00:00:00,W
4,6B,6,43,1992-02-12 00:00:00,W


CAMEO_DEU_2015 is categorical information. It is hard to use dummy variables as it has a large number of categories. Meanwhile, the major information about social class is also documented in CAMEO_DEUG_2015. Therefore, I decided to drop it. 

CAMEO_DEUG_2015 is ordinal information. I decided to convert it into integer. Notice that CAMEO_DEUG_2015 has 'X' for unknown. 

CAMEO_INTL_2015 has two information: the first number denotes family wealth, the second number denotes family composition. So I decided to split it into two columns. Notice that CAMEO_INTL_2015 has 'XX' for unknown. 

EINGEFUEGT_AM not much information as to what it is, so I decided to drop it. 

OST_WEST_KZ is converted to a dummy variable, as there is only West(W) and Other(O). If it's West, the value is 1; else 0.


In [16]:
def process_ob_col(df):
    '''
    It does data processing on the columns that have object as dtypes
    
    Input:
    df: original Dataframe;
    
    Output:
    processed DataFrame with only float/int values 
    '''
    #convert some remaining unknown cases into NaN 
    object_NaN_dict = {'CAMEO_DEUG_2015':['X'], 'CAMEO_INTL_2015': ['XX']}
    for col in list(object_NaN_dict.keys()):
        df.loc[:, col] = df.loc[:, col].apply(lambda x: np.nan if x in object_NaN_dict[col] else x)
        
    #drop two columns that do not have much additional information
    df_1 = df.drop(axis=1, columns=['CAMEO_DEU_2015', 'EINGEFUEGT_AM'], inplace=False)
    
    #convert Cameo_Deug_2015 into integers
    df_1.loc[:,'CAMEO_DEUG_2015'] = df_1.loc[:,'CAMEO_DEUG_2015'].apply(lambda x: float(x))
    
    #split CAMEO_INTL_2015 into two columns 
    df_1.loc[:,'CAMEO_INTL_2015'] = df_1.loc[:,'CAMEO_INTL_2015'].apply(lambda x: str(x))
    df_1.loc[:,'CAMEO_INTL_FAM_Wealth'] = [int(df_1['CAMEO_INTL_2015'].iloc[i][0]) if df_1['CAMEO_INTL_2015'].iloc[i] != 'nan' else np.nan for i in range(df_1.shape[0])]
    df_1.loc[:, 'CAMEO_INTL_FAM_COMPOSITION'] = [int(df_1['CAMEO_INTL_2015'].iloc[i][1]) if str(df_1['CAMEO_INTL_2015'].iloc[i]) != 'nan' else np.nan for i in range(df_1.shape[0])]

    #convert OST_WEST_KZ into a dummy variable
    OST_dict = {'W': 1, 'E':0}
    df_1['OST_WEST_KZ'] = df_1['OST_WEST_KZ'].map(OST_dict)
    
    df_2 = df_1.drop(axis=1, columns = 'CAMEO_INTL_2015', inplace = False)
    return df_2

In [17]:
azdias_process = process_ob_col(azdias_NaN_drop)

/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [18]:
azdias_process.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 278 entries, LNR to CAMEO_INTL_FAM_COMPOSITION
dtypes: float64(191), int64(87)
memory usage: 21.2 MB


1.5 Delete the rows with high percentage of NaN. Fill all NaN with column mean. 

In [19]:
def drop_row(df):
    """Drop the rows of the DataFrame that has over 15% NaN
       
       Input:original DataFrame; 
       
       Output:a transformed DataFrame that dropped high NaN rows"""
       
    thresh = int(0.85*df.shape[1])
    df_1 = df.dropna(axis=0, thresh=thresh, inplace=False)
    
    return df_1


In [20]:
azdias_drop_row = drop_row(azdias_process)

In [21]:
azdias_drop_row.head()

,LNR,AKT_DAT_KL,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,ANZ_KINDER,ANZ_PERSONEN,ANZ_STATISTISCHE_HAUSHALTE,ANZ_TITEL,ARBEIT,BALLRAUM,...,VK_DHT4A,VK_DISTANZ,VK_ZG11,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB,CAMEO_INTL_FAM_Wealth,CAMEO_INTL_FAM_COMPOSITION
1,910220,9.0,11.0,0.0,0.0,2.0,12.0,0.0,3.0,6.0,...,8.0,11.0,10.0,9.0,4.0,5,2,1,5.0,1.0
2,910225,9.0,10.0,0.0,0.0,1.0,7.0,0.0,3.0,2.0,...,9.0,9.0,6.0,9.0,2.0,5,2,3,2.0,4.0
3,910226,1.0,1.0,0.0,0.0,0.0,2.0,0.0,2.0,4.0,...,7.0,10.0,11.0,9.0,7.0,3,2,4,1.0,2.0
4,910241,1.0,3.0,0.0,0.0,4.0,3.0,0.0,4.0,2.0,...,3.0,5.0,4.0,9.0,3.0,4,1,3,4.0,3.0
5,910244,1.0,5.0,0.0,0.0,1.0,2.0,0.0,2.0,6.0,...,10.0,7.0,4.0,9.0,7.0,4,2,1,5.0,4.0


In [22]:
def fill_col_mean(df):
    """Fill the NaN with column mean
       
       Input: original DataFrame
       
       Output: transformed DataFrame that fills all NaN"""
    
    col_mean = df.mean()
    df_1 = df.fillna(col_mean, inplace=False)
    
    return df_1

In [23]:
azdias_finished = fill_col_mean(azdias_drop_row)

In [24]:
azdias_finished.isnull().any().any()

False

In [25]:
-1 in azdias_finished #make sure there is no unknown cases, which tend to be denoted by -1

False

1.6 Final Step for Data Cleaning: I created a function that collects all previous data cleaning procedures. So that for all the rest of DataFrames, I will just need to run this function on them for all preprocessing. 

In [26]:
def data_process(df):
    """All data cleaning process for the raw DataFrame from csv file 
       input: original dataframe 
       return: processed dataframe that has only integer/float as its values and no NaN"""
    df_1 = convert_unknown_to_NaN(df) #convert the unknown values to NaN
    df_2 = df_1.drop(axis=1, columns=column_NaN, inplace=False) #drop the columns that have high percentage of NaN
    df_3 = process_ob_col(df_2) #process the columns that have object dtypes 
    df_4 = drop_row(df_3) #drop the rows that have a high percentage of NaN 
    df_5 = fill_col_mean(df_4) #fill the remaining NaN with column means
    
    return df_5 
    

In [27]:
del azdias_1, azdias_NaN, azdias_NaN_drop, azdias_object, azdias_process, azdias_drop_row, azdias_finished

In [28]:
azdias_processed = data_process(azdias)

In [29]:
azdias_processed.head()

,LNR,AKT_DAT_KL,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,ANZ_KINDER,ANZ_PERSONEN,ANZ_STATISTISCHE_HAUSHALTE,ANZ_TITEL,ARBEIT,BALLRAUM,...,VK_DHT4A,VK_DISTANZ,VK_ZG11,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB,CAMEO_INTL_FAM_Wealth,CAMEO_INTL_FAM_COMPOSITION
1,910220,9.0,11.0,0.0,0.0,2.0,12.0,0.0,3.0,6.0,...,8.0,11.0,10.0,9.0,4.0,5,2,1,5.0,1.0
2,910225,9.0,10.0,0.0,0.0,1.0,7.0,0.0,3.0,2.0,...,9.0,9.0,6.0,9.0,2.0,5,2,3,2.0,4.0
3,910226,1.0,1.0,0.0,0.0,0.0,2.0,0.0,2.0,4.0,...,7.0,10.0,11.0,9.0,7.0,3,2,4,1.0,2.0
4,910241,1.0,3.0,0.0,0.0,4.0,3.0,0.0,4.0,2.0,...,3.0,5.0,4.0,9.0,3.0,4,1,3,4.0,3.0
5,910244,1.0,5.0,0.0,0.0,1.0,2.0,0.0,2.0,6.0,...,10.0,7.0,4.0,9.0,7.0,4,2,1,5.0,4.0


In [30]:
azdias_processed.isnull().any().any()

False

## PCA Model

2.1 Final minor adjustment to the data to facilitate the PCA model training. First I noticed that LNR is unique for each row and functions as an identifier for each individual. So I decided to drop it. Meanwhile, I want to normalize the data using a MinMaxScaler, so that the model can be trained faster. 

In [31]:
azdias_processed.drop(axis=1, columns = 'LNR', inplace=True)

In [32]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range= (0, 1)) #set the range to be between 0 and 1 for all values in the dataframe 

azdias_scaled= pd.DataFrame(scaler.fit_transform(azdias_processed.astype(float)))
azdias_scaled.index = azdias_processed.index
azdias_scaled.columns = azdias_processed.columns
azdias_scaled.head()


,AKT_DAT_KL,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,ANZ_KINDER,ANZ_PERSONEN,ANZ_STATISTISCHE_HAUSHALTE,ANZ_TITEL,ARBEIT,BALLRAUM,CAMEO_DEUG_2015,...,VK_DHT4A,VK_DISTANZ,VK_ZG11,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB,CAMEO_INTL_FAM_Wealth,CAMEO_INTL_FAM_COMPOSITION
1,1.0,0.018487,0.0,0.0,0.044444,0.026726,0.0,0.250,0.833333,0.875,...,0.7,0.833333,0.9,1.0,0.500,0.8,1.0,0.000,1.00,0.00
2,1.0,0.016807,0.0,0.0,0.022222,0.015590,0.0,0.250,0.166667,0.375,...,0.8,0.666667,0.5,1.0,0.250,0.8,1.0,0.250,0.25,0.75
3,0.0,0.001681,0.0,0.0,0.000000,0.004454,0.0,0.125,0.500000,0.125,...,0.6,0.750000,1.0,1.0,0.875,0.4,1.0,0.375,0.00,0.25
4,0.0,0.005042,0.0,0.0,0.088889,0.006682,0.0,0.375,0.166667,0.625,...,0.2,0.333333,0.3,1.0,0.375,0.6,0.0,0.250,0.75,0.50
5,0.0,0.008403,0.0,0.0,0.022222,0.004454,0.0,0.125,0.833333,0.875,...,0.9,0.500000,0.3,1.0,0.875,0.6,1.0,0.000,1.00,0.75


2.2 Define a PCA Model

In [33]:
import boto3
import sagemaker 
from sagemaker import get_execution_role

session = sagemaker.Session()
role = get_execution_role()
bucket_name = session.default_bucket()

In [34]:
prefix = 'Arvato_Project'

output_path='s3://{}/{}/'.format(bucket_name, prefix)

print('Training artifacts will be uploaded to: {}'.format(output_path))

Training artifacts will be uploaded to: s3://sagemaker-us-west-1-178050996200/Arvato_Project/


In [35]:
# define a PCA model
from sagemaker import PCA

N_COMPONENTS=276  #This is a rule of thumb: n-1, for starting the PCA model 

pca_SM = PCA(role=role,
             instance_count=1,
             instance_type='ml.m5.2xlarge',
             output_path=output_path,
             num_components=N_COMPONENTS, 
             sagemaker_session=session)

In [36]:
train_data_np = azdias_scaled.values.astype('float32')

In [37]:
# convert to RecordSet format
formatted_train_data = pca_SM.record_set(train_data_np)

2.3 Train the Model 

In [38]:
%%time

# train the PCA mode on the formatted data
pca_SM.fit(formatted_train_data) 

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2021-06-07 00:32:44 Starting - Starting the training job...
2021-06-07 00:32:47 Starting - Launching requested ML instancesProfilerReport-1623025964: InProgress
......
2021-06-07 00:34:13 Starting - Preparing the instances for training.........
2021-06-07 00:35:39 Downloading - Downloading input data
2021-06-07 00:35:39 Training - Downloading the training image..Docker entrypoint called with argument(s): train
Running default environment configuration script
[06/07/2021 00:35:56 INFO 139699241162560] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-conf.json: {'algorithm_mode': 'regular', 'subtract_mean': 'true', 'extra_components': '-1', 'force_dense': 'true', 'epochs': 1, '_log_level': 'info', '_kvstore': 'dist_sync', '_num_kv_servers': 'auto', '_num_gpus': 'auto'}
[06/07/2021 00:35:56 INFO 139699241162560] Merging with provided configuration from /opt/ml/input/config/hyperparameters.json: {'feature_dim': '277', 'num_components': 

2.4 Access Model Attributes.

In [39]:
training_job_name= 'pca-2021-06-07-00-32-44-462'

In [40]:
import os 
model_key = os.path.join(prefix, training_job_name, 'output/model.tar.gz')
#download and unzip the model 
boto3.resource('s3').Bucket(bucket_name).download_file(model_key, 'model.tar.gz')
os.system('tar -zxvf model.tar.gz')
os.system('unzip model_algo-1')

2304

In [41]:
import mxnet as mx 

In [42]:
pca_model_params = mx.ndarray.load('model_algo-1')

2.5 Pick the top n components that explain 80% variance in the data

In [43]:
s=pd.DataFrame(pca_model_params['s'].asnumpy()) #get the s value for each component
print(s)

               0
0            NaN
1       3.780533
2      11.522208
3      12.418921
4      15.376204
..           ...
271   815.508240
272   919.391479
273   963.828003
274  1116.289307
275  1366.424927

[276 rows x 1 columns]


In [44]:
def explained_variance(s, n_top_components):
    '''Calculates the approx. data variance that n_top_components captures.
       :param s: A dataframe of singular values for top components; 
           the top value is in the last row.
       :param n_top_components: An integer, the number of top components to use.
       :return: The expected data variance covered by the n_top_components.'''
    start_idx = N_COMPONENTS - n_top_components 
    sum = 0
    for i in range(start_idx, N_COMPONENTS):
        sum = sum + np.square(s.iloc[i, 0])
    exp_variance = sum / np.square(s).sum()
    
    #alternative: exp_variance = np.square(s.iloc[start_idx:,:]).sum()/np.square(s).sum()
    return exp_variance

In [49]:
#test how many components can capture 80% variance of the data

n_top_components = 80 # select a value for the number of top components

# calculate the explained variance
exp_variance = explained_variance(s, n_top_components)
print('Explained variance: ', exp_variance[0])

Explained variance:  0.8061778


Top 80 components can explain 80.6% data variance, which I think is sufficient. So I decided to reduce the features to top 80 components. 

2.5 Transform the dataframe to top 80 components to prepare for the K-Means Model. 

In [52]:
pca_predictor = pca_SM.deploy(initial_instance_count=1, 
                              instance_type='ml.m5.2xlarge')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


-----------!

In [53]:
train_pca = pca_predictor.predict(train_data_np) 

ClientError: An error occurred (413) when calling the InvokeEndpoint operation: 

In [ ]:
# create dimensionality-reduced data

def create_transformed_df(train_pca, azdias_scaled, n_top_components):
    ''' Return a dataframe of data points with component features. 
        The dataframe should contain component values.
        :param train_pca: A list of pca training data, returned by a PCA model.
        :param azdias_scaled: A dataframe of normalized, original features.
        :param n_top_components: An integer, the number of top components to use.
        :return: A dataframe with n_top_component values as columns.        
     '''
    # create a dataframe of component features, indexed by State-County
    azdias_transformed = pd.DataFrame(index=np.arange(azdias_scaled.shape[0]),columns=np.arange(n_top_components)) #specify the row number and column number
    for i in range(0, azdias_scaled.shape[0]):
        for j in range(0, n_top_components):
            component_list = [*train_pca[i].label['projection'].float32_tensor.values] #transform the iterables to a list
            azdias_transformed.iat[i,j] = component_list[N_COMPONENTS-1-j] #how to assign value using row and column index in DataFrame
            
    
    return azdias_transformed

In [ ]:
## Specify top n
top_n = 80

# call your function and create a new dataframe
azdias_transformed = create_transformed_df(train_pca, azdias_scaled, n_top_components=top_n)

column_names = []
for i in range(1, top_n+1):
    column_names.append('c_'+str(i)) 

azdias_transformed.columns = column_names

azdias_transformed.head()

In [54]:
#delete the endpoint
session.delete_endpoint(pca_predictor.endpoint) 

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


## K-Means 

## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter.

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.